## Import Dependencies and Open Instrument Handles

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_OSA_MS9740B import *

In [2]:
rm = pyvisa.ResourceManager()

osa = OSA_MS9740B()
osa.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [25]:
wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
Temperature = 23

Light_engine = "22-49-001"
Channel = 4

Bias_current_mA = 50
Voltage_V = 1.012

## Function Definition

In [26]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement Routine

In [38]:
meas_df = pd.DataFrame()
Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
meas_df_temp = pd.DataFrame({'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':Bias_current_mA, 'Voltage_V':Voltage_V, 'Temeprature_C':Temperature, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                            'Power_uW':Power_list_uW, 'Wavelength_peak_nm':Wavelength_peak_nm, 'Power_peak_nm':Power_peak_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                            '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
data = [meas_df_temp, meas_df]
meas_df = pd.concat(data,ignore_index=True,sort=False)

## Save Data

In [39]:
meas_df.to_csv(r"Channel_4_Bias_50_T_50.csv")

## Close Instrument Handles

In [3]:
osa.close()